In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2025-11-15 18:48:35.002184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 18:48:35.032842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-15 18:48:35.700588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def extract_surface_lines_np(brep_str):
    lines = brep_str.strip().split('\n')
    surface_stop = 0
    surface_line = 0

    for i, line in enumerate(lines):
        if line.startswith('Surface'):
            surface_line = i
        if line.startswith('Triangulations'):
            surface_stop = i
            break

    surfaces = []

    for i in range(surface_line + 1, surface_stop):
        surfaces.append(lines[i].strip().split(' '))

    surfaces_flat = []
    for i in surfaces:
        for j in i:
            surfaces_flat.append(float(j))

    return np.array(surfaces_flat, dtype=np.float32)

In [3]:
# test function
with open('models/cone/a cone with a 10.00mm diameter and a 20.00mm height.brep', 'r', encoding='utf-8') as f:
    b = f.read()
    print(extract_surface_lines_np(b))

[ 3.          0.          0.         20.          0.          0.
 -1.         -1.          0.         -0.          0.          1.
  0.          0.          0.24497867  1.          0.          0.
  0.          0.          0.         -1.         -1.          0.
 -0.          0.          1.          0.        ]


In [4]:
import os

shapes = ['cone', 'cube', 'cylinder', 'sphere']

file_list = [os.listdir('models/' + shape) for shape in shapes]

X = []
y = []

for shape in shapes:
    y.append([shape]*len(os.listdir('models/' + shape)))

# flatten
file_list = [item for items in file_list for item in items]
y = [item for items in y for item in items]

for i in range(0, len(file_list)):
    with open('models/' + y[i] + '/' + file_list[i], 'r', encoding='utf-8') as f:
        brep_str = f.read()
        X.append(extract_surface_lines_np(brep_str))

In [5]:
from numpy import float32
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

maxlen = 1024
X_pad = pad_sequences(X, maxlen=maxlen, dtype='float32', padding='post', truncating='post', value=0.0)
X_pad = X_pad[..., None]

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_arr = np.asarray(y_encoded, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_arr, test_size=0.2, random_state=3)

model = models.Sequential([
    layers.Input(shape=(maxlen, 1)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

W0000 00:00:1763250516.437460  237955 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model_trained = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=64)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4583 - loss: 1.4889 - val_accuracy: 0.8125 - val_loss: 0.5945
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7569 - loss: 0.5251 - val_accuracy: 1.0000 - val_loss: 0.2519
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.2315 - val_accuracy: 1.0000 - val_loss: 0.1167
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.1130 - val_accuracy: 1.0000 - val_loss: 0.0582
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0595 - val_accuracy: 1.0000 - val_loss: 0.0325
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0357 - val_accuracy: 1.0000 - val_loss: 0.0206
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0241 - val_accuracy: 1.0000 - val_loss: 0.0145
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0179 - val_accuracy: 1.0000 - val_loss: 0.0111
Epoch 9

# n-dim tensor?
# standard scaler?
# dense or convolutional?



Right now we have a working baseline model on the CAD shape data that Ross set up. Our focus is on how we check if these models are actually good,  before Confirming  “100% accuracy” without context.

We are keeping it simple and consistent: 80% of the data for training, 20% for testing. That way every new model we try can be compared under the same setup.

Here’s what were using to evaluate:

1. **Overall accuracy** – quick check of how many test shapes we got right. Good for sanity.
2. **Per-class accuracy** – how well we do on each shape type (cone, cube, cylinder, sphere) so we can see if the model is only good on one class.
3. **Confusion matrix** – to see which classes get mixed up with which. If two shapes are always confused, that tells us something about the features/model.

As we add more models or features, we can also plug in things like precision/recall or F1 if we see imbalance or want a fairer view across classes.